In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
import json

This is an example from https://www.freecodecamp.org/news/how-to-extract-keywords-from-text-with-tf-idf-and-pythons-scikit-learn-b2a0f3d7e667/

In [3]:
df_idf = pd.read_csv('amazon/reviews.csv')
df_dataset = pd.read_json('clothing_dataset/renttherunway_final_data.json', lines = True)

In [4]:
# print("Schema:\n", df_idf.dtypes)
# print("Shape of database =", df_idf.shape)

In [5]:
def pre_process(text):
    # to lowercase
    text=text.lower()
    
    # remove tags
    text = re.sub("&lt;/?.*?&gt;", "&lt;&gt; ", text)
    
    # remove special characters and digits
    text = re.sub("(\\d|\\W)+", " ", text)
    
    return text

def get_stop_words(stop_file_path):
    with open(stop_file_path, 'r', encoding='utf-8') as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return frozenset(stop_set)

In [6]:
df_idf['text'] = df_idf['title'] + " " + df_idf['body']
df_idf['text'] = df_idf['text'].apply(lambda x: pre_process(str(x)))

df_dataset['text'] = df_dataset['review_summary'] + " " + df_dataset['review_text']
df_dataset['text'] = df_dataset['text'].apply(lambda x: pre_process(str(x)))

sub_dataset = df_dataset[['text', 'rating']]

In [7]:
# print(df_idf['text'][2])
# print()
# print(sub_dataset['text'][0])

# print(df_idf['text'])


all_data = df_idf['text'].append(sub_dataset['text'])
print(len(sub_dataset['text']))

192544


In [8]:
stopwords = get_stop_words('stopwords.txt')
docs = all_data.tolist()

In [9]:
cv = CountVectorizer(max_df = .85, stop_words=stopwords)
wordCountVec = cv.fit_transform(docs)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [10]:
list(cv.vocabulary_.keys())[:10]

['def',
 'best',
 'worst',
 'samsung',
 'awhile',
 'absolute',
 'doo',
 'read',
 'review',
 'detect']

In [11]:
# tfidf = TfidfTransformer(smooth_idf=True, use_idf=True)
# tfidf.fit(wordCountVec)

In [12]:
# def sort_coo(coo_matrix):
#     tuples = zip(coo_matrix.col, coo_matrix.data)
#     return sorted(tuples, key=lambda x: (x[1] , x[0]), reverse=True)

# def extract_topn_from_vector(feature_names, sorted_items, topn=10):
#     sorted_items = sorted_items[:topn]
    
#     score_vals = []
#     feature_vals = []
    
#     for idx, score in sorted_items:
#         score_vals.append(round(score, 3))
#         feature_vals.append(feature_names[idx])
        
#         results = {}
#         for idx in range(len(feature_vals)):
#             results[feature_vals[idx]]=score_vals[idx]
            
#         return results

In [13]:
# feature_names = cv.get_feature_names()

# doc = docs[1]

# tf_idf_vector = tfidf.transform(cv.transform([doc]))

# sorted_items = sort_coo(tf_idf_vector.tocoo())

# keywords = extract_topn_from_vector(feature_names, sorted_items, 10)


In [17]:
# test = cv.transform([doc])

# print(keywords)
# for idx in range(len(sorted_items)):
#     print(feature_names[sorted_items[idx][0]], sorted_items[idx][1])
print(len(df_dataset))

192544


In [ ]:
y = df_dataset['rating']
# fixing the labels, if > 3.5 is going to be 1 which is positive, else 0
y = y.apply(lambda x: 1 if x > 5 else 0)#y.apply(lambda x: 1 if x > 3.5 else 0) 
y = y.to_numpy()
x = wordCountVec.toarray()[:len(df_dataset)]
print(x.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [15]:
# print(X_train[:, :], y_train[:])

In [16]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
# clf.fit(X_train, y_train)

In [17]:
# Save the model in a binary file
import pickle
filename = 'model2.sav'
pickle.dump(clf, open(filename, 'wb'))

In [18]:
# Loads the model from the binary file
import pickle
filename = 'model.sav'
clf = pickle.load(open(filename, 'rb'))

In [19]:
# print(X_test[:, 0], y_test[0])

In [20]:
# y_train_p = clf.predict(X_train)
# y_test_p = clf.predict(X_test)

In [21]:
# for i in range(len(y_test)):
#     print(y_test_p[i], y_test[i])

In [22]:
# from sklearn.metrics import accuracy_score, confusion_matrix
# print("Accuracy in testing set:", accuracy_score(y_test, y_test_p))
# print("Accuracy in training set:", accuracy_score(y_train, y_train_p))
# print(confusion_matrix(y_test, y_test_p))
# print(confusion_matrix(y_train, y_train_p))

In [23]:
test = cv.transform(["Hate", "Good", "Awful", "Best"]).toarray()
clf.predict(test)

array([0, 1, 0, 1])

In [24]:
# for i in range(110, 120):
#     test = cv.transform([docs[i]]).toarray()
#     p = clf.predict(test)
#     print(docs[i], p, y[i])
    
#     print(type(test))

# Adversarial Neural Network

In this section we start building the GANs, this model takes the word embedding and generate new embeddings that are similar to the given ones. 

In [30]:
def build_generator(img_shape):

    noise_shape = (100,)

    model = Sequential()

    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

def build_discriminator(shape):

    img_shape = shape

    model = Sequential()

#     model.add(Flatten(input_shape=img_shape)) # is one dimension
    model.add(Dense(512, input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)


In [31]:
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization, Activation, ZeroPadding2D

img_rows = 1
img_cols = X_train[0].shape
img_shape = (img_cols)

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

# Build and compile the generator
generator = build_generator(img_shape)
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# The generator takes noise as input and generated imgs
z = Input(shape=(100,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The valid takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 512)               17842688  
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 257       
Total params: 17,974,273
Trainable params: 17,974,273
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #  

In [32]:
def results(self, pred, actual):
    results = confusion_matrix(actual, pred)
    print('Confusion Matrix :')
    print(results)
    print ('Accuracy Score :',accuracy_score(actual, pred))
    print ('Report : ')
    print(classification_report(actual, pred))
    print()

In [33]:
def train(epochs, data, batch_size=128):


        # Rescale -1 to 1
        X_train = data #(X_train.astype(np.float32) - 127.5) / 127.5
#         X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[1], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = generator.predict(noise)

            # Train the discriminator
            d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))



In [34]:
train(epochs=100, data=X_train)

0 [D loss: 0.694669, acc.: 49.22%] [G loss: 0.963271]
1 [D loss: 0.781345, acc.: 68.75%] [G loss: 0.987514]
2 [D loss: 0.836146, acc.: 69.53%] [G loss: 1.056294]
3 [D loss: 1.161055, acc.: 58.59%] [G loss: 1.251272]
4 [D loss: 0.899115, acc.: 71.09%] [G loss: 1.401319]
5 [D loss: 0.882163, acc.: 65.62%] [G loss: 1.593820]
6 [D loss: 0.876900, acc.: 68.75%] [G loss: 1.877352]
7 [D loss: 0.729376, acc.: 71.88%] [G loss: 1.988602]
8 [D loss: 0.688198, acc.: 66.41%] [G loss: 2.270581]
9 [D loss: 0.700677, acc.: 62.50%] [G loss: 2.547071]
10 [D loss: 0.658148, acc.: 71.09%] [G loss: 2.873407]
11 [D loss: 0.478531, acc.: 75.00%] [G loss: 3.039404]
12 [D loss: 0.632129, acc.: 71.09%] [G loss: 3.033529]
13 [D loss: 0.485821, acc.: 76.56%] [G loss: 3.237605]
14 [D loss: 0.550982, acc.: 68.75%] [G loss: 3.539235]
15 [D loss: 0.498697, acc.: 77.34%] [G loss: 3.619682]
16 [D loss: 0.436810, acc.: 85.16%] [G loss: 3.434534]
17 [D loss: 0.494989, acc.: 85.16%] [G loss: 3.520747]
18 [D loss: 0.455619

In [35]:
print(X_train.shape)

(41407, 34848)


In [50]:
gen = 1
noise = np.random.normal(0, 1, (gen, 100))
new_mails = np.absolute(np.round(generator.predict(noise)))

idx = np.random.randint(0, X_train.shape[1], gen)
imgs = X_train[idx]

prediction = clf.predict(new_mails)

print("Predicted \t{}".format(new_mails))
print("Real \t\t{}".format(imgs))
print("prediction \t{}".format(prediction))

Predicted 	[[0. 0. 0. ... 0. 0. 0.]]
Real 		[[0 0 0 ... 0 0 0]]
prediction 	[0]


In [ ]:
test = cv.transform(["Hate", "Good", "Awful", "Best"]).toarray()
cv.inverse_transform(test) # See the generated words